In [1]:
from numpy.random import seed
import pandas as pd
import random as rn
import os

from TL4HDR.data.preProcess import get_one_race, get_n_years, standarize_dataset, get_dataset
from TL4HDR.examples.classify_util import run_mixture_cv, run_one_race_cv, \
    run_unsupervised_transfer_cv
from tensorflow import set_random_seed

Using TensorFlow backend.


In [2]:
seed(11111)
set_random_seed(11111)
os.environ['PYTHONHASHSEED'] = '0'
os.environ["KERAS_BACKEND"] = "tensorflow"
rn.seed(11111)

In [3]:
def run_cv(cancer_type, feature_type, target, years=3):
    print (cancer_type, feature_type, target, years)
    dataset = get_dataset(cancer_type=cancer_type, feature_type=feature_type, target=target, groups=("WHITE", "BLACK"))
    # dataset = read_data(cancer_type, feature_type[0], target, years)
    dataset = standarize_dataset(dataset)
    dataset_w = get_one_race(dataset, 'WHITE')
    dataset_w = get_n_years(dataset_w, years)
    dataset_b = get_one_race(dataset, 'BLACK')
    dataset_b = get_n_years(dataset_b, years)

    dataset = get_n_years(dataset, years)
    k = -1
    X, Y, R, y_sub, y_strat = dataset

    df = pd.DataFrame(y_strat, columns=['RY'])
    df['R'] = R
    df['Y'] = Y
    print(X.shape)
    print(df['RY'].value_counts())
    print(df['R'].value_counts())
    print(df['Y'].value_counts())


    parametrs_mix = {'fold': 3, 'k': k, 'val_size':0.0, 'batch_size':20,'momentum':0.9,
                     'learning_rate':0.015, 'lr_decay':0.03, 'dropout':0.5,
                     'L1_reg': 0.001, 'L2_reg': 0.001, 'hidden_layers': [128, 64]}
    parametrs_w = {'fold': 3, 'k': k, 'val_size':0.0, 'batch_size':20,
                     'learning_rate':0.02, 'lr_decay':0.03, 'dropout':0.5,
                     'L1_reg': 0.001, 'L2_reg': 0.001, 'hidden_layers': [128, 64]}
    parametrs_b = {'fold': 3, 'k': k, 'val_size':0.0, 'batch_size':4,
                     'learning_rate':0.01, 'lr_decay':0.0, 'dropout':0.5,
                     'L1_reg': 0.001, 'L2_reg': 0.001, 'hidden_layers': [128, 64]}
    parametrs_tl_sa = {'fold': 3, 'k': k, 'val_size':0.0, 'batch_size':20,
                     'learning_rate':0.005, 'lr_decay':0.0, 'dropout':0.5,
                     'L1_reg': 0.001, 'L2_reg': 0.001, 'hidden_layers': [128, 64]}

    res = pd.DataFrame()
    for i in range(20):
        seed = i
        df_m, _mixture_classifiers = run_mixture_cv(seed, dataset, **parametrs_mix)
        df_w, _w_classifiers = run_one_race_cv(seed, dataset_w, **parametrs_w)
        df_w = df_w.rename(columns={"Auc": "W_ind"})
        df_b, _b_classifiers = run_one_race_cv(seed, dataset_b, **parametrs_b)
        df_b = df_b.rename(columns={"Auc": "B_ind"})
        df_tl, _unsupervised_transfer_classifiers = run_unsupervised_transfer_cv(seed, dataset, **parametrs_tl_sa)

        df1 = pd.concat([df_m, df_w['W_ind'], df_b['B_ind'], df_tl['TL_Auc']],
                        sort=False, axis=1)
        print (df1)
        res = res.append(df1)

    f_name = 'TL4HDR/Result/' + cancer_type + '-AA-EA-' + feature_type[0] + '-' + target + '-' + str(years) + 'YR.xlsx'
    res.to_excel(f_name)

In [4]:
def main():
    run_cv('BRCA', 'Protein', 'OS', years=4)

In [ ]:
if __name__ == '__main__':
    main()

('BRCA', 'Protein', 'OS', 4)
(811, 190)
(810, 192)
(322, 189)
1WHITE    216
0WHITE     57
1BLACK     39
0BLACK     10
Name: RY, dtype: int64
WHITE    273
BLACK     49
Name: R, dtype: int64
1    255
0     67
Name: Y, dtype: int64


TL4HDR/model/mlp.py:158: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)
  theano.Param(corruption_level, default=0.2),
TL4HDR/model/mlp.py:159: UserWarning: The Param class is deprecated. Replace Param(default=N) by theano.In(value=N)
  theano.Param(learning_rate, default=0.1)


     A_Auc     B_Auc   W_Auc  folds     W_ind     B_ind   TL_Auc
0  0.70676  0.464103  0.7513      3  0.762102  0.571795  0.64359
     A_Auc     B_Auc     W_Auc  folds     W_ind     B_ind    TL_Auc
1  0.68405  0.479487  0.718405      3  0.719948  0.558974  0.694872
      A_Auc     B_Auc     W_Auc  folds     W_ind     B_ind    TL_Auc
2  0.636055  0.374359  0.681124      3  0.697612  0.589744  0.576923
